<a href="https://colab.research.google.com/github/vistec-AI/colab/blob/master/save_checkpoint_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

เริ่มจาก ต่อ Google Drive ก่อน.

ถ้าใช้วิธี save a copy จะ mount อัตโนมัติไม่ได้. ต้องสร้าง New notebook แล้ว copy-paste ไปทีละ cell

เปิด File Pane ด้านซ้าย
- เลือก Mount Drive
- คลิก Connect to Google Drive

Drive จะมาอยู่ที่ `/content/drive/My Drive/`

In [0]:
import tensorflow as tf

In [0]:
# โหลด data จาก CIFAR-10
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

170500096/170498071 [==============================] - 11s 0us/step


In [0]:
# สร้าง model CNN ง่ายๆ
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

def get_new_model():
    model = Sequential([
        Conv2D(filters=16, input_shape=(32, 32, 3), kernel_size=(3, 3), 
               activation='relu', name='conv_1'),
        Conv2D(filters=8, kernel_size=(3, 3), activation='relu', name='conv_2'),
        MaxPooling2D(pool_size=(4, 4), name='pool_1'),
        Flatten(name='flatten'),
        Dense(units=32, activation='relu', name='dense_1'),
        Dense(units=10, activation='softmax', name='dense_2')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [0]:
model = get_new_model()

In [0]:
# จะสร้าง callback เพื่อ save ทุกๆ epoch
from tensorflow.keras.callbacks import ModelCheckpoint
# เลือกให้ save ใน Google Drive เรา
checkpoint_path = '/content/drive/My Drive/checkpoint'
checkpoint = ModelCheckpoint(checkpoint_path, save_weights_only=True)

In [0]:
# ลอง train ง่ายๆ แค่ 3 epoch มันจะ save ทุกๆ epoch
model.fit(x=x_train, y=y_train, epochs=3, callbacks=[checkpoint])

Epoch 1/3
1563/1563 [==============================] - 49s 31ms/step - loss: 1.5953 - accuracy: 0.4229
Epoch 2/3
1563/1563 [==============================] - 49s 31ms/step - loss: 1.3158 - accuracy: 0.5348
Epoch 3/3
1563/1563 [==============================] - 49s 31ms/step - loss: 1.2332 - accuracy: 0.5644


สมมติ ว่า train เยอะๆ เช่น 10,000 epoch แล้ว Colab มันตัดไปซะก่อนครบ. เราก็ load จากของเดิม มา train ต่อได้

โดย run code เดิมทั้งหมดด้านบน ยกเว้น cell สุดท้าย (model.fit)

แล้วก็มา load model เก่าขึ้นมา train ต่อ

In [0]:
# ดูว่ามี file ที่ save ไว้จริง
!ls -al drive/"My Drive"/check*

-rw------- 1 root root     77 May 31 11:50 'drive/My Drive/checkpoint'
-rw------- 1 root root 177815 May 31 11:50 'drive/My Drive/checkpoint.data-00000-of-00001'
-rw------- 1 root root   2038 May 31 11:50 'drive/My Drive/checkpoint.index'


In [0]:
# load ของเก่า ขึ้นมา
model = get_new_model()
model.load_weights(checkpoint_path)

In [0]:
# train ต่ออีก 2 epoch เป็นตัวอย่าง
model.fit(x=x_train,
          y=y_train,
          epochs=2,
          callbacks=[checkpoint])

Epoch 1/2
1563/1563 [==============================] - 53s 34ms/step - loss: 1.1851 - accuracy: 0.5839
Epoch 2/2
1563/1563 [==============================] - 50s 32ms/step - loss: 1.1456 - accuracy: 0.5981


จะเห็นว่า accuracy เริ่มที่ 0.5 กว่าๆ คือเป็นการ train ต่อจากเดิม ไม่ได้เริ่ม train ใหม่จากศูนย์